In [1]:
import pandas as pd
import numpy as np

# Importando os dados já filtrados

In [2]:
cdr = pd.read_csv('/home/gustavo/Desktop/Mestrado/mestrado_dados/dados_18.11/cdr_after_filtes.csv')
cdr = cdr.drop('Unnamed: 0',1)
cdr.head()

,DATE,HOUR,DURATION,DDD_USER_FROM,USER_FROM,DDD_USER_TO,USER_TO,ANTENNA,9,10,11,12,13,14,DAY_OF_WEEK
0,2013-03-21,21:03:30,0.13,61,B3299B0E587D7275E3E4D530E9EECF50,98,6432F1DF21BA38368D9A165C739EEBB3,51601,195,LONGA DISTANCIA,BRT,OI,PRE PAGO,300000210,Thursday
1,2013-03-21,20:30:29,7.02,61,BA4DEA1F710C1216F74F1255150C49D6,61,4F6DCBA36292B8487743E13319E67DF6,51601,77,LOCAL,BRT,GVT,PRE PAGO,300000210,Thursday
2,2013-03-21,01:10:47,0.83,32,1AC8490BA99709F652B4A5C3525D3C82,32,B6BB8A38B286A41978E732BDE0249DB5,50141,117,LOCAL,OI,OI,POS PAGO,300000188,Thursday
3,2013-03-21,21:11:55,6.87,32,05AC98D91ED3C2821F53519B19CCBC58,32,A3ACD65D308B86F4487DB35A5F2E50EB,50141,117,LOCAL,OI,OI,PRE PAGO,300000533,Thursday
4,2013-03-21,20:29:21,0.13,61,BA4DEA1F710C1216F74F1255150C49D6,21,A49BA005920B8BA24BEE40F455B69B9B,51601,117,LOCAL,BRT,OI,PRE PAGO,300000210,Thursday


# Importando a localização das antenas

In [3]:
#É possível um mesmo conjunto de LAT e LONG terem mais de um Id atrelado
antennas = pd.read_csv('/home/gustavo/Desktop/Mestrado/mestrado_dados/dados_18.11/antenas_regiao_imediata_sjdr.csv', error_bad_lines = False, sep=';')
antennas = antennas.rename(columns={'id': 'ID', 'antenna': 'ANTENNA', 'city': 'CITY', 'lat': 'LAT', 'long':'LONG'})
antennas.head()

,ID,ANTENNA,CITY,LAT,LONG
0,4593,27712,Madre de Deus de Minas,-21.48194,-44.33039
1,4706,23442,São João del-Rei,-21.10707,-44.21979
2,4707,23463,São João del-Rei,-21.12699,-44.26144
3,4863,23462,São João del-Rei,-21.12699,-44.26144
4,5001,29943,Tiradentes,-21.11886,-44.16478


In [41]:
#O arquivo unique_locations contém as localizações únicas das antenas
#Atribui um identiifcador único pra elas, afim de adicionar esse identificador no dataframe de antenas
antennas_unique = pd.read_csv('/home/gustavo/Desktop/Mestrado/mestrado_dados/dados_18.11/unique_locations.txt', sep=']', header=None)
antennas_unique[0] = antennas_unique[0].str.replace('[','')
ids = ['MDM', 'SJDR1', 'SJDR2', 'SJDR3', 'SJDR4', 'SJDR5','TR1', 'TR2', 'LD1', 'RT', 'SVM', 'ST']
antennas_unique['ANTENNA_ID'] = ids
antennas_unique = antennas_unique.rename(columns={0:'ANTENNA', 1: 'CITY'})
antennas_unique.head(13)

,ANTENNA,CITY,ANTENNA_ID
0,"27711,27712,27713,48582,27451,48581",Madre de Deus de Minas,MDM
1,"23441,23442,50153,23443,50152,50151",São João del-Rei,SJDR1
2,"23461,23463,23462,50933,50931,50932",São João del-Rei,SJDR2
3,"23471,23472,23473,51311,51312",São João del-Rei,SJDR3
4,"23451,23452,23453,50663,50661,50662",São João del-Rei,SJDR4
5,"50141,50142,50143",São João del-Rei,SJDR5
6,"29941,29943,53562,50911,49131,50913,29942,5356...",Tiradentes,TR1
7,"29011,29013,29012",Tiradentes,TR2
8,"49592,51041,51042,49593,51043",Lagoa Dourada,LD1
9,"51601,51603,51602",Ritápolis,RT


In [42]:
antennas_with_id = antennas.merge(antennas_unique.assign(ANTENNA=antennas_unique['ANTENNA'].str.split(','))
               .explode('ANTENNA').assign(ANTENNA=lambda x: x['ANTENNA'].astype(int)), on='ANTENNA')

antennas_with_id = antennas_with_id.drop('CITY_y', 1)
antennas_with_id = antennas_with_id.rename(columns={'CITY_x': 'CITY'})
antennas_with_id.head()

,ID,ANTENNA,CITY,LAT,LONG,ANTENNA_ID
0,4593,27712,Madre de Deus de Minas,-21.48194,-44.33039,MDM
1,4706,23442,São João del-Rei,-21.10707,-44.21979,SJDR1
2,4707,23463,São João del-Rei,-21.12699,-44.26144,SJDR2
3,4863,23462,São João del-Rei,-21.12699,-44.26144,SJDR2
4,5001,29943,Tiradentes,-21.11886,-44.16478,TR1


In [43]:
cdr_merge_antennas = pd.merge(cdr, antennas_with_id[['ANTENNA', 'ANTENNA_ID']], on='ANTENNA', how='left')
cdr_merge_antennas.head(7)

,DATE,HOUR,DURATION,DDD_USER_FROM,USER_FROM,DDD_USER_TO,USER_TO,ANTENNA,9,10,11,12,13,14,DAY_OF_WEEK,ANTENNA_ID
0,2013-03-21,21:03:30,0.13,61,B3299B0E587D7275E3E4D530E9EECF50,98,6432F1DF21BA38368D9A165C739EEBB3,51601,195,LONGA DISTANCIA,BRT,OI,PRE PAGO,300000210,Thursday,RT
1,2013-03-21,20:30:29,7.02,61,BA4DEA1F710C1216F74F1255150C49D6,61,4F6DCBA36292B8487743E13319E67DF6,51601,77,LOCAL,BRT,GVT,PRE PAGO,300000210,Thursday,RT
2,2013-03-21,01:10:47,0.83,32,1AC8490BA99709F652B4A5C3525D3C82,32,B6BB8A38B286A41978E732BDE0249DB5,50141,117,LOCAL,OI,OI,POS PAGO,300000188,Thursday,SJDR5
3,2013-03-21,21:11:55,6.87,32,05AC98D91ED3C2821F53519B19CCBC58,32,A3ACD65D308B86F4487DB35A5F2E50EB,50141,117,LOCAL,OI,OI,PRE PAGO,300000533,Thursday,SJDR5
4,2013-03-21,20:29:21,0.13,61,BA4DEA1F710C1216F74F1255150C49D6,21,A49BA005920B8BA24BEE40F455B69B9B,51601,117,LOCAL,BRT,OI,PRE PAGO,300000210,Thursday,RT
5,2013-03-21,00:09:27,0.13,61,023CF68D76C38796F6C3B138F27AC657,21,A49BA005920B8BA24BEE40F455B69B9B,51601,117,LOCAL,BRT,OI,PRE PAGO,300000210,Thursday,RT
6,2013-03-21,22:36:30,1.22,61,B046BF714E16102B098297E89639BA30,61,A943CD0FB95D587FF5D10F61D643AD73,51601,117,LOCAL,BRT,BRT,PRE PAGO,300000210,Thursday,RT


In [44]:
presumed_residence = pd.read_csv('/home/gustavo/Desktop/Mestrado/mestrado_dados/Regiao_Presumida/residence_antenna.csv', sep=';')
presumed_residence = presumed_residence.rename(columns={'user': 'USER','residence_antenna': 'ANTENNA'})
presumed_residence_with_id = pd.merge(presumed_residence, antennas_with_id[['ANTENNA', 'ANTENNA_ID']], 
                                      on='ANTENNA', how='left')

presumed_residence_with_id = presumed_residence_with_id.rename(columns={'USER': 'USER_FROM', 
                                                                        'ANTENNA_ID': 'PRESUMED_RESIDENCE'})
presumed_residence_with_id

,USER_FROM,ANTENNA,PRESUMED_RESIDENCE
0,00003F26576D1122848B6F3CEEC250E5,50141,SJDR5
1,000579E8BA1DA9E5292392473BCCCD2A,49592,LD1
2,0008962B966EF6C5571DF03F5C786677,23461,SJDR2
3,0008B7FB336065A53EC31AD06D712E28,51601,RT
4,0008B8E32E18BF4450B7CAAF0C1CE179,23441,SJDR1
...,...,...,...
59785,FFFA3339BA362F788F81DF22F3F7B6BA,27711,MDM
59786,FFFDA183352D8DD3AC2913B09BD4174F,23451,SJDR4
59787,FFFDBC5EE75F3127C262488987D6F99D,23441,SJDR1
59788,FFFE9C91DE90C5CA962FE90319E8D620,23451,SJDR4


In [45]:
merged = pd.merge(cdr_merge_antennas, presumed_residence_with_id[['USER_FROM', 'PRESUMED_RESIDENCE']], on='USER_FROM', how='right')
merged.sample(15)

,DATE,HOUR,DURATION,DDD_USER_FROM,USER_FROM,DDD_USER_TO,USER_TO,ANTENNA,9,10,11,12,13,14,DAY_OF_WEEK,ANTENNA_ID,PRESUMED_RESIDENCE
498199,2013-04-14,13:27:58,0.13,61.0,EB125DA353A8459843EEE64FBE441A55,61.0,053483A340810687DA3683AE70EBEAA3,51601.0,117.0,LOCAL,BRT,BRT,PRE PAGO,300000210.0,Sunday,RT,RT
531874,2013-03-28,20:15:17,1.17,82.0,FAB62467F6779C45078BB10DA84BF774,82.0,C899EDAD6073C5417C37B4DDD99E1383,23451.0,117.0,LOCAL,OI,OI,PRE PAGO,300000210.0,Thursday,SJDR4,MDM
187694,2013-03-31,18:06:14,0.38,82.0,5969CB03F01024EFF6A95F86EFE61C42,82.0,EF334DC82E19FB5C64B7FB358EB3FAF1,23451.0,117.0,LOCAL,OI,OI,PRE PAGO,300000210.0,Sunday,SJDR4,SJDR4
344113,2013-03-30,21:48:04,0.68,82.0,A30ACC851BC2278D2EDBD167FF618192,82.0,6AF09E20B4876E7C2E956CA137ECC076,23451.0,117.0,LOCAL,OI,OI,POS PAGO,300000292.0,Saturday,SJDR4,SJDR4
126935,2013-04-07,16:38:55,10.80,82.0,3C2EE0608373223AF183436F614BC611,82.0,A6366EC97A6904CA1A742CF9A19AF19E,23451.0,117.0,LOCAL,OI,OI,PRE PAGO,300000210.0,Sunday,SJDR4,SJDR4
154217,2013-03-23,19:08:06,5.08,83.0,496FBD8EEDABBDB718F238008A61513A,83.0,22915220F9C8DC6EDCA6F2B1BA3BDE27,23441.0,117.0,LOCAL,OI,OI,PRE PAGO,300000210.0,Saturday,SJDR1,SJDR1
348205,2013-03-24,22:21:48,0.15,82.0,A4EC2B5C071ECAAEECE0D9849F16F22F,82.0,B8D06C6954039680CCF8EF0BE2A1998F,23471.0,117.0,LOCAL,OI,OI,PRE PAGO,300000210.0,Sunday,SJDR3,SJDR3
255301,2013-04-07,19:58:10,0.08,84.0,7928FBAB17BF99D35A64846B7F1E312C,84.0,8C90466AC3418735324932CAAD868068,60761.0,117.0,LOCAL,OI,OI,PRE PAGO,300000140.0,Sunday,SVM,SVM
229067,2013-04-07,08:50:32,0.45,83.0,6C90D8B5D4ED595ED9005C2A5A59E6DC,83.0,CE49967DEF7D7B5E465AE403D06E4721,23441.0,117.0,LOCAL,OI,OI,PRE PAGO,300000210.0,Sunday,SJDR1,SJDR1
207983,2013-03-31,11:58:17,0.32,91.0,62C09CEBCC650B3001F1C28E9E0BA1C1,91.0,AFE8C56D7906E03FCCC4A8762882049D,23451.0,117.0,LOCAL,OI,OI,PRE PAGO,300000210.0,Sunday,SJDR4,SJDR4


In [46]:
merged = merged.rename(columns={'ANTENNA_ID': 'CALL_LOC'})
merged.sample(15)

,DATE,HOUR,DURATION,DDD_USER_FROM,USER_FROM,DDD_USER_TO,USER_TO,ANTENNA,9,10,11,12,13,14,DAY_OF_WEEK,CALL_LOC,PRESUMED_RESIDENCE
299168,2013-03-23,23:25:57,4.52,61.0,8D92A89416A0405E7626739788A1FF2B,61.0,4419AA907077F48310750B75C083ABA4,51601.0,117.0,LOCAL,BRT,BRT,PRE PAGO,300000210.0,Saturday,RT,RT
243531,2013-04-01,19:47:41,1.55,82.0,7351E80B1EB3469ECBD58091E7FA5E43,82.0,F3066E239E584145AA516BD897FB7608,23451.0,117.0,LOCAL,OI,OI,PRE PAGO,300000533.0,Monday,SJDR4,SJDR4
103955,2013-03-21,19:57:58,1.02,82.0,30E250E6DA2890FBA1A5F69D74EFE78D,82.0,34B68909672E999158C0E1147EB237CD,23461.0,117.0,LOCAL,OI,OI,PRE PAGO,300000210.0,Thursday,SJDR2,SJDR2
463948,2013-04-08,22:06:45,5.58,91.0,DB0CA265ADCBE2F2C487B2E025A6E475,91.0,A50615689D8873A9BC44BA8D7F85431A,23441.0,77.0,LOCAL,OI,TNL,PRE PAGO,300000210.0,Monday,SJDR1,SJDR1
353622,2013-03-24,20:38:26,3.57,83.0,A74C8BB25D918FE25161953C1E009101,83.0,E4C2F8F405A4CFA72478CF0BDDBD0177,50141.0,117.0,LOCAL,OI,OI,POS PAGO,300000563.0,Sunday,SJDR5,SJDR5
344614,2013-04-17,21:21:28,0.07,82.0,A338A257E792B2EB4AB26A9410814790,82.0,2DD681ECAC46005BE78F5EAC25D5F901,23451.0,117.0,LOCAL,OI,OI,PRE PAGO,300000210.0,Wednesday,SJDR4,SJDR4
124827,2013-03-25,23:00:17,0.08,82.0,3B16FAA45AB9C5638381948D0AE14060,21.0,A49BA005920B8BA24BEE40F455B69B9B,23441.0,117.0,LOCAL,OI,OI,PRE PAGO,300000210.0,Monday,SJDR1,SJDR1
277544,2013-04-16,21:28:35,4.50,83.0,833CA957F73E205C1A345B7EC6513504,83.0,772FE101CD22694A6B40C03ECC2F9A73,23441.0,117.0,LOCAL,OI,OI,PRE PAGO,300000210.0,Tuesday,SJDR1,SJDR1
427603,2013-03-31,19:44:00,0.72,82.0,CA3718D93C4640E40BF80366519ABF1D,82.0,13BB05B958F66E2D0BB9073DC516D7D6,23461.0,117.0,LOCAL,OI,OI,POS PAGO,300000197.0,Sunday,SJDR2,SJDR2
528375,2013-04-16,19:44:21,0.08,91.0,F9273F184D01F8B275DB2761A9865736,91.0,A520468863F031948C7611B713089477,23451.0,117.0,LOCAL,OI,OI,PRE PAGO,300000210.0,Tuesday,SJDR4,SJDR4


In [47]:
grouped = merged.groupby('USER_FROM').agg({'PRESUMED_RESIDENCE': min, 'CALL_LOC': list })
grouped.sample(10)

,PRESUMED_RESIDENCE,CALL_LOC
USER_FROM,,
5DE8F13044EF0D14DBA826704CCC78A9,SJDR4,"[SJDR4, SJDR4, SJDR4, SJDR4, SJDR4, SJDR4, SJD..."
A6F938EDB552F40D6EDA4977636F1A12,SJDR4,"[SJDR4, SJDR4, SJDR3, SJDR4, SJDR4, SJDR4, SJD..."
9A8D6E9E528EEEA370180F63994CFDF1,SVM,"[SVM, SVM, SVM, SVM, SVM, SVM, SVM, SVM, SVM, ..."
32962828F3995D51A28B67C780575204,LD1,"[SJDR4, LD1, SJDR4, LD1, LD1, LD1, LD1, LD1, L..."
ECD52AF7729A68CDBA6B95B3F31B71D6,SJDR1,"[SJDR1, SJDR1]"
1FB45B4F8AA5762190D861B2423090FB,SJDR1,"[SJDR1, SJDR1, SJDR1, SJDR1, SJDR1, SJDR1, SJDR5]"
E9829F4F04A29E896DB53B14DDDFF0A6,SJDR5,"[SJDR5, SJDR5, SJDR5, SJDR5, SJDR5, SJDR5, SJD..."
AC1C03937D7E74E8C133E715BDD4A1D6,SJDR4,"[SJDR4, SJDR4]"
93F6AB7E8EE5353FAD9F00CC36C1A5EE,SJDR4,"[SJDR4, SJDR4, SJDR4, SJDR4, SJDR4]"


In [48]:
grouped['CALL_LOC'] = [[z for z in x if z!=y]
                       for x,y in zip(grouped['CALL_LOC'],
                                     grouped['PRESUMED_RESIDENCE'])]

grouped.sample(15)

,PRESUMED_RESIDENCE,CALL_LOC
USER_FROM,,
369B3B4EC3576C516141E36D39B251B0,SVM,[]
831BDA1AD57E0074D7EFDBC89321EB5B,RT,[]
D3380BC21F08014582A4313D01BE143F,SJDR3,[]
628AEC129D19834B56360649A4019DB9,SJDR2,[]
48F15D3175B07F5522EB2E3ADA331F84,LD1,[]
89ABAEB20AE8EAE6C6991F61CD76AEC9,SJDR1,[]
5E59D61E206B3EFAED2F2E231A0254AD,SVM,[SJDR4]
57F353FEFBB8210C8F660D45351256CC,LD1,[]
FEA551EEDF1593DDC2DE2C26CD75A8E4,SJDR4,[SJDR1]


In [49]:
grouped.to_csv('/home/gustavo/Desktop/Mestrado/mestrado_dados/dados_18.11/presumed_res_and_call_loc.csv')
antennas_with_id.to_csv('/home/gustavo/Desktop/Mestrado/mestrado_dados/dados_18.11/antenas_with_id.csv')